In [4]:
!pip install opencv-python albumentations pillow numpy scikit-learn

In [14]:
import cv2
import numpy as np
import os
import zipfile
import tempfile
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw
import albumentations as A
import random
import math
import shutil
from sklearn.cluster import KMeans

class FaceAugmentationPipeline:
    def __init__(self, data_source, output_path, auto_cleanup=True):
        self.data_source = data_source
        self.output_path = output_path
        self.auto_cleanup = auto_cleanup
        self.target_size = (256, 256)
        self.temp_dir = None

        # Check if data_source is a ZIP file or a directory
        if os.path.isfile(data_source) and data_source.lower().endswith('.zip'):
            self.data_path = self._extract_zip()
        elif os.path.isdir(data_source):
            self.data_path = data_source
        else:
            raise ValueError(f"Invalid data source: {data_source}. Must be a ZIP file or directory.")

        # Initialize face detector
        self.init_face_detector()

        # Create output directory
        os.makedirs(output_path, exist_ok=True)

        # Define augmentation pipeline
        self.augmentations = self.create_augmentation_pipeline()

    def _extract_zip(self):
        """Extract ZIP file to a temporary directory"""
        # Create temporary directory
        self.temp_dir = tempfile.mkdtemp()
        print(f"Extracting ZIP file to temporary directory: {self.temp_dir}")

        try:
            with zipfile.ZipFile(self.data_source, 'r') as zip_ref:
                # Extract all files
                zip_ref.extractall(self.temp_dir)

                # Find the 'data' folder inside the extracted content
                extracted_items = os.listdir(self.temp_dir)

                # Look for 'data' folder
                for item in extracted_items:
                    item_path = os.path.join(self.temp_dir, item)
                    if os.path.isdir(item_path) and item.lower() == 'data':
                        print(f"Found data folder: {item_path}")
                        return item_path

                # If no 'data' folder found, check if the ZIP contains the person folders directly
                person_folders = [item for item in extracted_items
                                 if os.path.isdir(os.path.join(self.temp_dir, item))]

                if person_folders:
                    print("No 'data' folder found. Using extracted directory directly.")
                    return self.temp_dir
                else:
                    raise ValueError("No data or person folders found in the ZIP file.")

        except zipfile.BadZipFile:
            raise ValueError(f"Invalid ZIP file: {self.data_source}")
        except Exception as e:
            if self.temp_dir and os.path.exists(self.temp_dir):
                shutil.rmtree(self.temp_dir)
            raise e
        """Initialize face detection model"""
        # Try to load Haar cascade
        try:
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.detector_type = 'haar'
            print("Using Haar cascade face detector")
        except:
            print("Haar cascade not found. Please install opencv-python properly.")
            self.detector_type = None

    def init_face_detector(self):
        """Initialize face detection model"""
        # Try to load Haar cascade
        try:
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.detector_type = 'haar'
            print("Using Haar cascade face detector")
        except:
            print("Haar cascade not found. Please install opencv-python properly.")
            self.detector_type = None

    def detect_face(self, image):
        """Detect and crop face from image"""
        if self.detector_type == 'haar':
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )

            if len(faces) > 0:
                # Get the largest face
                (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])

                # Add some margin around face
                margin = int(0.3 * max(w, h))
                x_start = max(0, x - margin)
                y_start = max(0, y - margin)
                x_end = min(image.shape[1], x + w + margin)
                y_end = min(image.shape[0], y + h + margin)

                face_crop = image[y_start:y_end, x_start:x_end]
                return face_crop

        return None

    def resize_image(self, image, target_size=(256, 256)):
        """Resize image to target size while maintaining aspect ratio"""
        h, w = image.shape[:2]

        # Calculate the aspect ratio
        aspect_ratio = w / h
        target_aspect_ratio = target_size[0] / target_size[1]

        if aspect_ratio > target_aspect_ratio:
            # Width is larger relative to height
            new_width = target_size[0]
            new_height = int(target_size[0] / aspect_ratio)
        else:
            # Height is larger relative to width
            new_height = target_size[1]
            new_width = int(target_size[1] * aspect_ratio)

        # Resize image
        resized = cv2.resize(image, (new_width, new_height))

        # Create a square image with padding
        square_image = np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)

        # Calculate padding
        pad_x = (target_size[0] - new_width) // 2
        pad_y = (target_size[1] - new_height) // 2

        # Place resized image in center
        square_image[pad_y:pad_y+new_height, pad_x:pad_x+new_width] = resized

        return square_image

    def create_augmentation_pipeline(self):
        """Create comprehensive augmentation pipeline"""
        augmentations = {
            # Geometric Transformations
            'rotation': A.Rotate(limit=20, p=1.0),
            'horizontal_flip': A.HorizontalFlip(p=1.0),
            'zoom': 'manual',  # Will implement manually for zoom in/out
            'translation': A.Affine(translate_percent=(0.1, 0.15), p=1.0),

            # Color/Lighting Augmentations
            'brightness': A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0, p=1.0),
            'contrast': A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=0.2, p=1.0),
            'saturation': A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=15, val_shift_limit=0, p=1.0),
            'hue_shift': A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=0, val_shift_limit=0, p=1.0),
            'gaussian_noise': A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),

            # Advanced Augmentations
            'gaussian_blur': A.GaussianBlur(blur_limit=(1, 3), p=1.0),
            'sharpen': A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0),
            'random_crop': 'manual',  # Implement manually for compatibility

            # Additional 8 Augmentations (including background change)
            'perspective': A.Perspective(scale=(0.05, 0.1), p=1.0),
            'elastic_transform': A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=1, p=1.0),
            'affine': A.Affine(shear=(-10, 10), p=1.0),
            'color_jitter': A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1.0),
            'shadow_highlight': 'manual',  # Will implement manually
            'clahe': A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
            'motion_blur': A.MotionBlur(blur_limit=7, p=1.0),
            'background_change': 'manual',  # NEW: Background replacement
        }

        return augmentations

    def apply_shadow_highlight(self, image):
        """Manually apply shadow/highlight effect"""
        # Convert to PIL for easier manipulation
        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Create a random elliptical mask for shadow/highlight
        width, height = pil_image.size
        mask = Image.new('L', (width, height), 0)

        # Random ellipse position and size
        x = random.randint(0, width//3)
        y = random.randint(0, height//3)
        w = random.randint(width//3, width)
        h = random.randint(height//3, height)

        # Draw ellipse on mask
        draw = ImageDraw.Draw(mask)
        draw.ellipse([x, y, x+w, y+h], fill=255)

        # Apply blur to mask for smooth transition
        mask = mask.filter(ImageFilter.GaussianBlur(radius=20))

        # Create highlight or shadow effect
        if random.choice([True, False]):
            # Highlight effect
            bright_image = ImageEnhance.Brightness(pil_image).enhance(1.3)
            result = Image.composite(bright_image, pil_image, mask)
        else:
            # Shadow effect
            dark_image = ImageEnhance.Brightness(pil_image).enhance(0.7)
            result = Image.composite(dark_image, pil_image, mask)

        # Convert back to OpenCV format
        return cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR)

    def apply_zoom(self, image):
        """Manually apply zoom effect (zoom in or out)"""
        h, w = image.shape[:2]

        # Random zoom factor between 0.8 and 1.2
        zoom_factor = random.uniform(0.8, 1.2)

        if zoom_factor == 1.0:
            return image

        # Calculate new dimensions
        new_w = int(w * zoom_factor)
        new_h = int(h * zoom_factor)

        if zoom_factor > 1.0:
            # Zoom in (crop from center)
            resized = cv2.resize(image, (new_w, new_h))
            start_x = (new_w - w) // 2
            start_y = (new_h - h) // 2
            cropped = resized[start_y:start_y+h, start_x:start_x+w]
            return cropped
        else:
            # Zoom out (add padding)
            resized = cv2.resize(image, (new_w, new_h))
            # Create a canvas with original size
            canvas = np.zeros((h, w, 3), dtype=np.uint8)
            # Calculate padding
            pad_x = (w - new_w) // 2
            pad_y = (h - new_h) // 2
            # Place resized image on canvas
            canvas[pad_y:pad_y+new_h, pad_x:pad_x+new_w] = resized
    def apply_background_change(self, image):
        """Replace background with different patterns or colors"""
        # Convert to PIL for easier processing
        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        h, w = image.shape[:2]

        # Simple background removal using color clustering
        # This is a simplified approach - for better results, use models like U2-Net or REMBG
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        data = image_rgb.reshape((-1, 3))

        # Use K-means to find dominant colors (assuming background is one of them)
        kmeans = KMeans(n_clusters=3, random_state=42)
        kmeans.fit(data)

        # Get cluster centers and labels
        colors = kmeans.cluster_centers_
        labels = kmeans.labels_

        # Reshape labels back to image shape
        labels = labels.reshape((h, w))

        # Identify background cluster (usually the largest cluster)
        unique, counts = np.unique(labels, return_counts=True)
        background_cluster = unique[np.argmax(counts)]

        # Create mask for background
        background_mask = (labels == background_cluster).astype(np.uint8) * 255

        # Apply morphological operations to clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_CLOSE, kernel)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_OPEN, kernel)

        # Apply Gaussian blur to soften edges
        background_mask = cv2.GaussianBlur(background_mask, (15, 15), 0)

        # Convert mask to 3-channel
        background_mask_3ch = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR) / 255.0

        # Generate new background
        bg_type = random.choice(['solid', 'gradient', 'pattern'])

        if bg_type == 'solid':
            # Solid color background
            colors = [
                [255, 255, 255],  # White
                [0, 0, 0],        # Black
                [240, 240, 240],  # Light gray
                [50, 50, 50],     # Dark gray
                [240, 248, 255],  # Alice blue
                [255, 250, 240],  # Floral white
            ]
            color = random.choice(colors)
            new_background = np.full((h, w, 3), color, dtype=np.uint8)

        elif bg_type == 'gradient':
            # Gradient background
            gradient = np.zeros((h, w, 3), dtype=np.uint8)

            # Random gradient direction and colors
            direction = random.choice(['horizontal', 'vertical', 'diagonal'])
            color1 = [random.randint(200, 255), random.randint(200, 255), random.randint(200, 255)]
            color2 = [random.randint(100, 200), random.randint(100, 200), random.randint(100, 200)]

            if direction == 'horizontal':
                for x in range(w):
                    ratio = x / w
                    gradient[:, x] = [
                        color1[0] * (1 - ratio) + color2[0] * ratio,
                        color1[1] * (1 - ratio) + color2[1] * ratio,
                        color1[2] * (1 - ratio) + color2[2] * ratio
                    ]
            elif direction == 'vertical':
                for y in range(h):
                    ratio = y / h
                    gradient[y, :] = [
                        color1[0] * (1 - ratio) + color2[0] * ratio,
                        color1[1] * (1 - ratio) + color2[1] * ratio,
                        color1[2] * (1 - ratio) + color2[2] * ratio
                    ]
            else:  # diagonal
                for y in range(h):
                    for x in range(w):
                        ratio = (x + y) / (w + h)
                        gradient[y, x] = [
                            color1[0] * (1 - ratio) + color2[0] * ratio,
                            color1[1] * (1 - ratio) + color2[1] * ratio,
                            color1[2] * (1 - ratio) + color2[2] * ratio
                        ]
            new_background = gradient

        else:  # pattern
            # Create simple patterns
            pattern_type = random.choice(['dots', 'lines', 'checker'])

            if pattern_type == 'dots':
                new_background = np.full((h, w, 3), [240, 240, 240], dtype=np.uint8)
                # Add random dots
                for _ in range(50):
                    x = random.randint(0, w-20)
                    y = random.randint(0, h-20)
                    cv2.circle(new_background, (x, y), random.randint(5, 15), [220, 220, 220], -1)

            elif pattern_type == 'lines':
                new_background = np.full((h, w, 3), [250, 250, 250], dtype=np.uint8)
                # Add vertical lines
                for x in range(0, w, 40):
                    cv2.line(new_background, (x, 0), (x, h), [230, 230, 230], 2)

            else:  # checker
                new_background = np.full((h, w, 3), [245, 245, 245], dtype=np.uint8)
                # Create checkerboard pattern
                square_size = 30
                for y in range(0, h, square_size):
                    for x in range(0, w, square_size):
                        if (x // square_size + y // square_size) % 2 == 0:
                            new_background[y:y+square_size, x:x+square_size] = [235, 235, 235]

        # Blend original image with new background using the mask
        result = image * (1 - background_mask_3ch) + new_background * background_mask_3ch
        result = result.astype(np.uint8)

        return result

    def apply_random_crop(self, image):
        """Manually apply random crop and resize back"""
        h, w = image.shape[:2]

        # Random scale between 0.8 and 1.0
        scale = random.uniform(0.8, 1.0)

        # Calculate crop size
        crop_h = int(h * scale)
        crop_w = int(w * scale)

        # Random crop position
        start_x = random.randint(0, w - crop_w)
        start_y = random.randint(0, h - crop_h)

        # Crop the image
        cropped = image[start_y:start_y+crop_h, start_x:start_x+crop_w]

        # Resize back to original size
        resized = cv2.resize(cropped, (w, h))

        return resized

    def apply_background_change(self, image):
        """Replace background with different patterns or colors"""
        # Convert to PIL for easier processing
        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        h, w = image.shape[:2]

        # Simple background removal using color clustering
        # This is a simplified approach - for better results, use models like U2-Net or REMBG
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        data = image_rgb.reshape((-1, 3))

        # Use K-means to find dominant colors (assuming background is one of them)
        kmeans = KMeans(n_clusters=3, random_state=42)
        kmeans.fit(data)

        # Get cluster centers and labels
        colors = kmeans.cluster_centers_
        labels = kmeans.labels_

        # Reshape labels back to image shape
        labels = labels.reshape((h, w))

        # Identify background cluster (usually the largest cluster)
        unique, counts = np.unique(labels, return_counts=True)
        background_cluster = unique[np.argmax(counts)]

        # Create mask for background
        background_mask = (labels == background_cluster).astype(np.uint8) * 255

        # Apply morphological operations to clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_CLOSE, kernel)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_OPEN, kernel)

        # Apply Gaussian blur to soften edges
        background_mask = cv2.GaussianBlur(background_mask, (15, 15), 0)

        # Convert mask to 3-channel
        background_mask_3ch = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR) / 255.0

        # Generate new background
        bg_type = random.choice(['solid', 'gradient', 'pattern'])

        if bg_type == 'solid':
            # Solid color background
            colors = [
                [255, 255, 255],  # White
                [0, 0, 0],        # Black
                [240, 240, 240],  # Light gray
                [50, 50, 50],     # Dark gray
                [240, 248, 255],  # Alice blue
                [255, 250, 240],  # Floral white
            ]
            color = random.choice(colors)
            new_background = np.full((h, w, 3), color, dtype=np.uint8)

        elif bg_type == 'gradient':
            # Gradient background
            gradient = np.zeros((h, w, 3), dtype=np.uint8)

            # Random gradient direction and colors
            direction = random.choice(['horizontal', 'vertical', 'diagonal'])
            color1 = [random.randint(200, 255), random.randint(200, 255), random.randint(200, 255)]
            color2 = [random.randint(100, 200), random.randint(100, 200), random.randint(100, 200)]

            if direction == 'horizontal':
                for x in range(w):
                    ratio = x / w
                    gradient[:, x] = [
                        color1[0] * (1 - ratio) + color2[0] * ratio,
                        color1[1] * (1 - ratio) + color2[1] * ratio,
                        color1[2] * (1 - ratio) + color2[2] * ratio
                    ]
            elif direction == 'vertical':
                for y in range(h):
                    ratio = y / h
                    gradient[y, :] = [
                        color1[0] * (1 - ratio) + color2[0] * ratio,
                        color1[1] * (1 - ratio) + color2[1] * ratio,
                        color1[2] * (1 - ratio) + color2[2] * ratio
                    ]
            else:  # diagonal
                for y in range(h):
                    for x in range(w):
                        ratio = (x + y) / (w + h)
                        gradient[y, x] = [
                            color1[0] * (1 - ratio) + color2[0] * ratio,
                            color1[1] * (1 - ratio) + color2[1] * ratio,
                            color1[2] * (1 - ratio) + color2[2] * ratio
                        ]
            new_background = gradient

        else:  # pattern
            # Create simple patterns
            pattern_type = random.choice(['dots', 'lines', 'checker'])

            if pattern_type == 'dots':
                new_background = np.full((h, w, 3), [240, 240, 240], dtype=np.uint8)
                # Add random dots
                for _ in range(50):
                    x = random.randint(0, w-20)
                    y = random.randint(0, h-20)
                    cv2.circle(new_background, (x, y), random.randint(5, 15), [220, 220, 220], -1)

            elif pattern_type == 'lines':
                new_background = np.full((h, w, 3), [250, 250, 250], dtype=np.uint8)
                # Add vertical lines
                for x in range(0, w, 40):
                    cv2.line(new_background, (x, 0), (x, h), [230, 230, 230], 2)

            else:  # checker
                new_background = np.full((h, w, 3), [245, 245, 245], dtype=np.uint8)
                # Create checkerboard pattern
                square_size = 30
                for y in range(0, h, square_size):
                    for x in range(0, w, square_size):
                        if (x // square_size + y // square_size) % 2 == 0:
                            new_background[y:y+square_size, x:x+square_size] = [235, 235, 235]

        # Blend original image with new background using the mask
        result = image * (1 - background_mask_3ch) + new_background * background_mask_3ch
        result = result.astype(np.uint8)

        return result

    def augment_image(self, image, augmentation_name):
        """Apply specific augmentation to image"""
        if augmentation_name == 'shadow_highlight':
            return self.apply_shadow_highlight(image)

        if augmentation_name == 'zoom':
            return self.apply_zoom(image)

        if augmentation_name == 'random_crop':
            return self.apply_random_crop(image)

        if augmentation_name == 'background_change':
            return self.apply_background_change(image)

        if augmentation_name not in self.augmentations:
            return image

        # For albumentations transforms
        if self.augmentations[augmentation_name] != 'manual':
            transform = A.Compose([self.augmentations[augmentation_name]])
            augmented = transform(image=image)
            return augmented['image']

        return image

    def process_person_folder(self, person_folder):
        """Process all images in a person's folder"""
        person_name = os.path.basename(person_folder)
        last_name = person_name.split('-')[-1].lower()

        # Create output directory for this person
        person_output_dir = os.path.join(self.output_path, person_name)
        os.makedirs(person_output_dir, exist_ok=True)

        # Get all image files
        image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
        image_files = [f for f in os.listdir(person_folder)
                      if f.lower().endswith(image_extensions)]

        print(f"Found {len(image_files)} images in {person_name} folder")

        for img_file in image_files:
            img_path = os.path.join(person_folder, img_file)

            # Read image
            image = cv2.imread(img_path)
            if image is None:
                print(f"Could not read image: {img_path}")
                continue

            # Get base filename without extension
            base_name = os.path.splitext(img_file)[0]
            file_ext = os.path.splitext(img_file)[1]

            # Detect and crop face
            face_crop = self.detect_face(image)
            if face_crop is None:
                print(f"No face detected in {img_path}. Using full image.")
                face_crop = image

            # Resize to target size
            resized_image = self.resize_image(face_crop, self.target_size)

            # Save original with same name
            original_filename = f"{base_name}_original{file_ext}"
            cv2.imwrite(os.path.join(person_output_dir, original_filename), resized_image)
            print(f"Saved original: {original_filename}")

            # Apply augmentations to this specific image
            for aug_name in self.augmentations.keys():
                try:
                    # Ensure the image is valid before processing
                    if resized_image is None or resized_image.size == 0:
                        print(f"Skipping {aug_name} for {img_file}: invalid image")
                        continue

                    augmented_image = self.augment_image(resized_image, aug_name)

                    # Check if augmentation produced valid result
                    if augmented_image is None or augmented_image.size == 0:
                        print(f"Skipping {aug_name} for {img_file}: augmentation produced empty image")
                        continue

                    # Generate filename: originalname_augmentation.ext
                    aug_filename = f"{base_name}_{aug_name}{file_ext}"
                    output_path = os.path.join(person_output_dir, aug_filename)

                    # Save augmented image
                    success = cv2.imwrite(output_path, augmented_image)
                    if success:
                        print(f"Saved: {aug_filename}")
                    else:
                        print(f"Failed to save: {aug_filename}")
                except Exception as e:
                    print(f"Error applying {aug_name} to {img_file}: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    continue

    def cleanup_temp_files(self):
        """Clean up temporary extracted files"""
        if self.temp_dir and os.path.exists(self.temp_dir):
            print(f"Cleaning up temporary directory: {self.temp_dir}")
            shutil.rmtree(self.temp_dir)
            self.temp_dir = None

    def process_all_folders(self):
        """Process all person folders in the dataset"""
        try:
            print("Starting face detection and augmentation pipeline...")

            # Get all person folders
            person_folders = [f for f in os.listdir(self.data_path)
                             if os.path.isdir(os.path.join(self.data_path, f))]

            total_folders = len(person_folders)
            for i, folder in enumerate(person_folders, 1):
                print(f"\nProcessing {folder} ({i}/{total_folders})...")
                folder_path = os.path.join(self.data_path, folder)
                self.process_person_folder(folder_path)

            print("\nAugmentation pipeline completed!")
            print(f"Results saved in: {self.output_path}")

            # Print summary
            total_images = 0
            total_expected_per_person = 5 * 20  # 5 original images × 20 versions each
            for person_folder in person_folders:
                person_output_dir = os.path.join(self.output_path, person_folder)
                if os.path.exists(person_output_dir):
                    num_images = len([f for f in os.listdir(person_output_dir)
                                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    total_images += num_images
                    print(f"{person_folder}: {num_images} images (expected: {total_expected_per_person})")

            print(f"\nTotal images generated: {total_images}")
            print(f"Expected total: {len(person_folders) * total_expected_per_person}")
            print("(5 images × 20 versions per person × 10 people)")

        finally:
            # Clean up temporary files if auto_cleanup is enabled
            if self.auto_cleanup:
                self.cleanup_temp_files()

# Usage example
if __name__ == "__main__":
    # Update these paths to match your setup
    DATA_SOURCE = "./data.zip"  # Path to your ZIP file (or directory)
    OUTPUT_PATH = "./augmented_data"  # Where to save augmented images

    # Create and run the pipeline
    # Set auto_cleanup=False if you want to keep the extracted files for later use
    pipeline = FaceAugmentationPipeline(DATA_SOURCE, OUTPUT_PATH, auto_cleanup=True)
    pipeline.process_all_folders()

    def download_results(self, zip_name="augmented_data_results"):
        """Create zip file and download results (for Colab/cloud environments)"""
        zip_filename = f"{zip_name}.zip"

        print(f"\nCreating ZIP file: {zip_filename}")
        shutil.make_archive(zip_name, 'zip', self.output_path)

        # Check if we're in Google Colab
        try:
            from google.colab import files
            print("Downloading ZIP file...")
            files.download(zip_filename)
            print("Download started!")
        except ImportError:
            print(f"ZIP file created locally: {zip_filename}")
            print("You can manually download this file from your file explorer.")

        return zip_filename

Extracting ZIP file to temporary directory: /tmp/tmpvk19xbxh
Found data folder: /tmp/tmpvk19xbxh/data
Using Haar cascade face detector
Starting face detection and augmentation pipeline...

Processing Barack-Obama (1/10)...
Found 5 images in Barack-Obama folder


<ipython-input-14-c6215e8f3308>:169: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  'gaussian_noise': A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
<ipython-input-14-c6215e8f3308>:178: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  'elastic_transform': A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=1, p=1.0),


Saved original: down_original.jpg
Saved: down_rotation.jpg
Saved: down_horizontal_flip.jpg
Skipping zoom for down.jpg: augmentation produced empty image
Saved: down_translation.jpg
Saved: down_brightness.jpg
Saved: down_contrast.jpg
Saved: down_saturation.jpg
Saved: down_hue_shift.jpg
Saved: down_gaussian_noise.jpg
Saved: down_gaussian_blur.jpg
Saved: down_sharpen.jpg
Saved: down_random_crop.jpg
Saved: down_perspective.jpg
Saved: down_elastic_transform.jpg
Saved: down_affine.jpg
Saved: down_color_jitter.jpg
Saved: down_shadow_highlight.jpg
Saved: down_clahe.jpg
Saved: down_motion_blur.jpg
Saved: down_background_change.jpg
Saved original: left_original.jpg
Saved: left_rotation.jpg
Saved: left_horizontal_flip.jpg
Saved: left_zoom.jpg
Saved: left_translation.jpg
Saved: left_brightness.jpg
Saved: left_contrast.jpg
Saved: left_saturation.jpg
Saved: left_hue_shift.jpg
Saved: left_gaussian_noise.jpg
Saved: left_gaussian_blur.jpg
Saved: left_sharpen.jpg
Saved: left_random_crop.jpg
Saved: left_

In [1]:
import shutil

# Zip the output folder
shutil.make_archive('augmented_data', 'zip', './augmented_data')

# Download the zip file
from google.colab import files
files.download('augmented_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>